# Eclat Algorithm Implementation Showcase

This is a small toy example to test and showcase my (simple) implementation of the eclat algorithm for frequent item set mining. 

In [1]:
import pandas as pd
import numpy as np
import fpm_with_eclat.tiny_eclat as teclat # functions used for eclat based item set mining

## Simulate toy data

To get some toy data, we simulate item frequencies for a "shopping basket in a very (very) small supermarket".

In [ ]:
np.random.seed(42)
trials, df_length = 1, 100
my_transaction_df = pd.DataFrame({"Apple": np.random.binomial(n = trials, p = 0.7, size = df_length),
                                  "Grapes": np.random.binomial(n = trials, p = .4, size = df_length),
                                  "Eggs": np.random.binomial(n = trials, p = .5, size = df_length),
                                  "Cheese": np.random.binomial(n = trials, p = .6, size = df_length),
                                  "Lego": np.random.binomial(n = trials, p = .2, size = df_length)})
my_transaction_df["Wine"] = my_transaction_df["Grapes"].values * my_transaction_df["Cheese"].values  # deterministic: if someone buys grapes & cheese, they also buy wine!

# Binary transaction database (as df)
my_transaction_df.head(10)

,Apple,Grapes,Eggs,Cheese,Lego,Wine
0,1,1,0,0,0,0
1,0,1,1,1,0,1
2,0,1,0,1,0,1
3,1,1,1,1,1,1
4,1,0,1,0,0,0
5,1,1,0,1,0,1
6,1,0,0,0,0,0
7,0,0,1,0,0,0
8,1,0,0,1,0,0
9,0,0,0,0,0,0


In [10]:
# Transform it to as list of sets
my_transaction_db = teclat.transform_pandas_to_t(my_transaction_df)
print(teclat.transform_pandas_to_t.__doc__)
print(my_transaction_db[:10])


    Transforms a binary transaction database into a list of sets.

    Args:
        df (pandas.DataFrame): a dense transaction database with items as columns, rows as transactions, and int or bool entries indicating the existence (True | 1) or absence (False | 0) of an item in the corresponding transaction (i.e., row).

    Returns:
        a list of sets

    
[{'Apple', 'Grapes'}, {'Wine', 'Cheese', 'Eggs', 'Grapes'}, {'Wine', 'Cheese', 'Grapes'}, {'Lego', 'Eggs', 'Cheese', 'Apple', 'Wine', 'Grapes'}, {'Apple', 'Eggs'}, {'Apple', 'Cheese', 'Wine', 'Grapes'}, {'Apple'}, {'Eggs'}, {'Apple', 'Cheese'}, set()]


In [11]:
# Aggregate our transaction list of sets to one transaction dict containing information about item support (i.e., frequency) and positional indices of their occurences in the list of sets.
my_transaction_dict = teclat.get_transaction_dict(transaction_db = my_transaction_db)
print(teclat.get_transaction_dict.__doc__)
print(my_transaction_dict)


    Transforms our transaction list (of sets) to a (sorted) dictionary containing the frequency per item and the indices of its occurences in our transaction list.

    Args:
      transaction_db (list): a list of sets
      sort_asc (bool): should our transaction dictionary be sorted ascendingly

    Returns:
      a nested dictionary containing frequency and (positional) indices per item
    
{'Lego': {'frequency': 11, 'positions': {3, 40, 41, 42, 45, 47, 48, 49, 19, 26, 28}}, 'Wine': {'frequency': 13, 'positions': {1, 2, 3, 5, 37, 38, 44, 17, 19, 23, 25, 30, 31}}, 'Grapes': {'frequency': 21, 'positions': {0, 1, 2, 3, 5, 12, 17, 19, 20, 23, 24, 25, 26, 30, 31, 36, 37, 38, 41, 42, 44}}, 'Eggs': {'frequency': 23, 'positions': {1, 3, 4, 7, 12, 13, 14, 15, 16, 18, 19, 20, 21, 26, 27, 29, 34, 36, 37, 39, 40, 46, 47}}, 'Cheese': {'frequency': 27, 'positions': {1, 2, 3, 5, 8, 10, 13, 14, 16, 17, 18, 19, 21, 22, 23, 25, 27, 29, 30, 31, 34, 37, 38, 39, 40, 44, 45}}, 'Apple': {'frequency': 39

In [12]:
# Run the eclat algorithm
relative_minimal_support = .2
absolute_minimal_support = int(relative_minimal_support*len(my_transaction_db))
fpm_results = teclat.tiny_eclat(transaction_dict = my_transaction_dict, smin = minimal_support)
print(teclat.tiny_eclat.__doc__)
print(fpm_results)


  The recursive Eclat algorithm.

  On highest level, we input all singleton items with their support in ascending order.
  For each singleton we check, if it meets the minimal support. 
  If so, we save this observation in our output dictionary F. 
  Then, we go into the recursion, i.e., we define our current frequent element as prefix and build a conditional transaction dict from it.
  This conditional transaction dict is then processed as usual with tiny_eclat().

  Args:
    transaction_dict (dict): our sorted (conditional) transaction dictionary.
    smin (int): our minimal absolut support (i.e., frequency to determine "frequent items"); can also be written as relative support [%] * N_transactions.
    prefix (str): empty in the highest recursion; gets extended when conditional transaction dictionaries are built.
    F (dict): our interim output dictionary for frequent items

  Returns:
    F (dict): a nested dictionary containing all (frequent) item sets, with their support (i.e

In [13]:
fpm_summary = teclat.summarise_tiny_eclat(fpm_results, as_df = True)
print(teclat.summarise_tiny_eclat.__doc__)
print(fpm_summary.head())


  Summarise output of tiny_eclat(), by calculating the frequent item set size, and descendingly sorting the resulting dictionary based on 1) frequent item set size and 2) support (i.e., set frequency).
  If not differently specified, the output is formatted as a pandas dataframe.

  Args:
    tiny_eclat_results (dict): output dictionary of tiny_eclat(), containing frequent item sets (i.e., meeting minimal support) with their support and positional indices.
    as_df (bool): if True the output dictionary is formated as a pandas dataframe

  Results:
    either a sorted dictionary or dataframe
  
             KEY         ITEM_SET  ITEM_SET_SIZE  SUPPORT  \
0  Cheese__Apple  {Apple, Cheese}              2       23   
1          Apple          {Apple}              1       39   
2         Cheese         {Cheese}              1       27   
3           Eggs           {Eggs}              1       23   
4         Grapes         {Grapes}              1       21   

                              